In [1]:
import pandas as pd
import numpy as np
import random
import os

In [2]:
gpa_map = {
    'A+': 4.3, 'A': 4.0, 'A-': 3.7,
    'B+': 3.3, 'B': 3.0, 'B-': 2.7,
    'C+': 2.3, 'C': 2.0, 'C-': 1.7,
    'D+': 1.3, 'D': 1.0
}

In [3]:
def get_random_grade():
    grades = list(gpa_map.keys())
    probabilities = [0.02, 0.08, 0.1, 0.2, 0.3, 0.1, 0.05, 0.05, 0.05, 0.025, 0.025]
    grade = np.random.choice(grades, p=probabilities)
    return grade, gpa_map[grade]

In [4]:
def generate_student_data(student_type):
    grades = [-1] * 50
    years = [-1] * 50
    
    if student_type == "undergraduate":
        course_range = range(6, 13)
        year_range = range(1, 5)
    else:  # graduate
        course_range = range(6, 11)
        year_range = range(5, 7)
        
    for year in year_range:
        num_courses = random.choice(course_range)
        taken_courses = random.sample([i for i, y in enumerate(years) if y == -1], num_courses)
        
        for course_pos in taken_courses:
            grades[course_pos] = get_random_grade()[1]
            years[course_pos] = year
            
        # Terminate randomly
        if random.choice([True, False]):
            break

        
    return grades, years

In [5]:
grades_data = []
years_data = []
for i in range(1, 101):
    if i <= 70:  # Undergraduate
        grades, years = generate_student_data("undergraduate")
    else:  # Graduate
        grades, years = generate_student_data("graduate")
    grades_data.append(grades)
    years_data.append(years)


In [6]:
# for student in grades_df.index:
#     num_courses = random.randint(4, 20)
#     taken_courses = random.sample(list(grades_df.columns), num_courses)
    
#     for course in taken_courses:
#         grades_df.iat[student-1, course-1] = get_random_grade()[1]
        
#         if student <= 4: # for undergrad
#             years_df.iat[student-1, course-1] = random.randint(1, 4)
#         else: # for grad students
#             years_df.iat[student-1, course-1] = random.randint(5, 6)

In [7]:
grades_df = pd.DataFrame(grades_data, index=range(1, 101), columns=range(1, 51))
years_df = pd.DataFrame(years_data, index=range(1, 101), columns=range(1, 51))

In [8]:
grades_df.head(), years_df.head()

(    1    2    3    4    5    6    7    8    9    10  ...   41   42   43   44  \
 1 -1.0 -1.0 -1.0  3.7 -1.0 -1.0 -1.0 -1.0  2.3  3.0  ... -1.0 -1.0 -1.0 -1.0   
 2 -1.0 -1.0  2.3  3.0 -1.0 -1.0 -1.0  3.3  3.3 -1.0  ... -1.0 -1.0 -1.0 -1.0   
 3 -1.0 -1.0 -1.0  3.3  3.7 -1.0  3.0  2.7  3.3 -1.0  ... -1.0  2.3 -1.0  3.0   
 4  3.0  3.0 -1.0  4.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0 -1.0   
 5  2.3  1.3 -1.0 -1.0  3.7  1.3  2.0  3.7 -1.0  2.0  ... -1.0  1.7  3.0 -1.0   
 
     45   46   47   48   49   50  
 1  3.3 -1.0 -1.0 -1.0 -1.0 -1.0  
 2  2.3 -1.0  2.3 -1.0  3.0 -1.0  
 3  3.0 -1.0 -1.0 -1.0 -1.0  3.0  
 4 -1.0 -1.0 -1.0 -1.0 -1.0  3.3  
 5  3.7  3.7  2.7  3.0  3.3  3.3  
 
 [5 rows x 50 columns],
    1   2   3   4   5   6   7   8   9   10  ...  41  42  43  44  45  46  47  \
 1  -1  -1  -1   1  -1  -1  -1  -1   1   1  ...  -1  -1  -1  -1   1  -1  -1   
 2  -1  -1   2   1  -1  -1  -1   1   2  -1  ...  -1  -1  -1  -1   1  -1   1   
 3  -1  -1  -1   1   1  -1   2   2   2 

# save to sql & load back

In [9]:
import sqlite3
from sqlalchemy import create_engine

In [10]:
if not os.path.exists("db"):
    os.mkdir("db")
engine = create_engine('sqlite:///db/student_data.sql')
grades_df.to_sql('grades', con=engine, if_exists='replace', index_label='student')
years_df.to_sql('years_taken', con=engine, if_exists='replace', index_label='student')


100

In [11]:
engine = create_engine('sqlite:///db/student_data.sql')

loaded_grades_df = pd.read_sql_table('grades', con=engine, index_col='student')
loaded_years_df = pd.read_sql_table('years_taken', con=engine, index_col='student')

loaded_grades_df.head(), loaded_years_df.head()


(           1    2    3    4    5    6    7    8    9   10  ...   41   42   43  \
 student                                                    ...                  
 1       -1.0 -1.0 -1.0  3.7 -1.0 -1.0 -1.0 -1.0  2.3  3.0  ... -1.0 -1.0 -1.0   
 2       -1.0 -1.0  2.3  3.0 -1.0 -1.0 -1.0  3.3  3.3 -1.0  ... -1.0 -1.0 -1.0   
 3       -1.0 -1.0 -1.0  3.3  3.7 -1.0  3.0  2.7  3.3 -1.0  ... -1.0  2.3 -1.0   
 4        3.0  3.0 -1.0  4.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  ... -1.0 -1.0 -1.0   
 5        2.3  1.3 -1.0 -1.0  3.7  1.3  2.0  3.7 -1.0  2.0  ... -1.0  1.7  3.0   
 
           44   45   46   47   48   49   50  
 student                                     
 1       -1.0  3.3 -1.0 -1.0 -1.0 -1.0 -1.0  
 2       -1.0  2.3 -1.0  2.3 -1.0  3.0 -1.0  
 3        3.0  3.0 -1.0 -1.0 -1.0 -1.0  3.0  
 4       -1.0 -1.0 -1.0 -1.0 -1.0 -1.0  3.3  
 5       -1.0  3.7  3.7  2.7  3.0  3.3  3.3  
 
 [5 rows x 50 columns],
          1  2  3  4  5  6  7  8  9  10  ...  41  42  43  44  45  46  47  48

# Recommendation Test

In [18]:
second_year_students = years_df.apply(lambda row: set(row.unique()) == {-1, 1,2}, axis=1)
second_year_student_id = second_year_students.index[second_year_students].tolist()[1]
selected_student_data = years_df.loc[second_year_student_id]

In [22]:
total_courses_taken = sum(selected_student_data != -1)
courses_per_year = dict(selected_student_data.value_counts())
del courses_per_year[-1]
courses_per_year, total_courses_taken # 10 courses in first year and 12 courses in second year.

({2: 12, 1: 10}, 22)

In [39]:
reshaped_year = years_df.stack().reset_index()
reshaped_year.columns = ["student_id", "course_id", "year"]
reshaped_year = reshaped_year[reshaped_year['year'] != -1]

In [40]:
reshaped_grades = grades_df.stack().reset_index()
reshaped_grades.columns = ["student_id", "course_id", "grades"]
reshaped_grades = reshaped_grades[reshaped_grades['grades'] != -1]

In [41]:
merged_data = reshaped_year.merge(reshaped_grades, on=['student_id', 'course_id'])

In [42]:
merged_data

,student_id,course_id,year,grades
0,1,4,1,3.7
1,1,9,1,2.3
2,1,10,1,3.0
3,1,16,1,3.0
4,1,23,1,2.7
...,...,...,...,...
1527,100,35,6,3.0
1528,100,37,5,3.0
1529,100,44,6,3.3
1530,100,47,6,2.3


In [46]:
matrix_df = merged_data.pivot(index='student_id', columns='course_id', values='grades')
matrix_df

course_id,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
student_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,3.7,NaN,NaN,NaN,NaN,2.3,3.0,...,NaN,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2.3,3.0,NaN,NaN,NaN,3.3,3.3,NaN,...,NaN,NaN,NaN,NaN,2.3,NaN,2.3,NaN,3.0,NaN
3,NaN,NaN,NaN,3.3,3.7,NaN,3.0,2.7,3.3,NaN,...,NaN,2.3,NaN,3.0,3.0,NaN,NaN,NaN,NaN,3.0
4,3.0,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.3
5,2.3,1.3,NaN,NaN,3.7,1.3,2.0,3.7,NaN,2.0,...,NaN,1.7,3.0,NaN,3.7,3.7,2.7,3.0,3.3,3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4.0,3.3,NaN,NaN,4.0,NaN,NaN,NaN,NaN,2.7,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,NaN,...,3.3,NaN,NaN,3.3,NaN,3.3,NaN,NaN,NaN,NaN
98,NaN,1.0,NaN,2.0,NaN,3.3,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,3.0,2.7,3.0,3.7,NaN,NaN


In [48]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df = matrix_df.fillna(0)

cosine_sim = cosine_similarity(df)

cosine_df = pd.DataFrame(cosine_sim, index=df.index, columns=df.index)




In [51]:
target_student = second_year_student_id

similar_students = cosine_df[target_student].sort_values(ascending=False)

In [53]:
similar_students # similarity score for student_idx = 3

student_id
3     1.000000
69    0.675636
5     0.631235
33    0.608185
52    0.599844
        ...   
37    0.117058
79    0.097026
16    0.083729
28    0.075646
61    0.000000
Name: 3, Length: 100, dtype: float64

In [55]:
num_similar_students = 5# topN
top_similar_students = similar_students.drop(target_student).head(num_similar_students).index

# Identify courses the target student hasn't taken
target_courses = df.loc[target_student]
courses_not_taken = target_courses[target_courses == 0].index

course_recommendations = {}

for course in courses_not_taken:
    avg_score = df.loc[top_similar_students, course].mean()
    course_recommendations[course] = avg_score

sorted_recommendations = {course: score for course, score in sorted(course_recommendations.items(), key=lambda item: item[1], reverse=True)}

for course, expected_gpa in sorted_recommendations.items():
    print(f"Course: {course}, Expected GPA: {expected_gpa:.2f}")



Course: 34, Expected GPA: 3.32
Course: 6, Expected GPA: 3.00
Course: 37, Expected GPA: 2.92
Course: 19, Expected GPA: 2.82
Course: 48, Expected GPA: 2.74
Course: 21, Expected GPA: 2.72
Course: 30, Expected GPA: 2.72
Course: 47, Expected GPA: 2.68
Course: 22, Expected GPA: 2.60
Course: 24, Expected GPA: 2.52
Course: 31, Expected GPA: 2.52
Course: 46, Expected GPA: 2.48
Course: 14, Expected GPA: 2.32
Course: 29, Expected GPA: 2.20
Course: 43, Expected GPA: 2.20
Course: 2, Expected GPA: 2.00
Course: 12, Expected GPA: 2.00
Course: 10, Expected GPA: 1.88
Course: 1, Expected GPA: 1.80
Course: 35, Expected GPA: 1.80
Course: 40, Expected GPA: 1.80
Course: 18, Expected GPA: 1.72
Course: 49, Expected GPA: 1.66
Course: 15, Expected GPA: 1.40
Course: 23, Expected GPA: 1.40
Course: 41, Expected GPA: 0.94
Course: 3, Expected GPA: 0.66
Course: 11, Expected GPA: 0.54


In [61]:
def recommend_courses(df, target_student, topN_student=5, topN_courses=5):
    top_similar_students = df.drop(target_student).head(topN_student).index
    target_courses = df.loc[target_student]
    courses_not_taken = target_courses[target_courses == 0].index
    
    course_recommendations = {}

    for course in courses_not_taken:
        avg_score = df.loc[top_similar_students, course].mean()
        course_recommendations[course] = avg_score

    sorted_recommendations = {course: score for course, score in sorted(course_recommendations.items(), key=lambda item: item[1], reverse=True)}
    return dict(list(sorted_recommendations.items())[:topN_courses])



In [67]:
recommendations = recommend_courses(df, target_student=2, topN_student=5, topN_courses=5)
for course, expected_gpa in recommendations.items():
    print(f"Course: {course}, Expected GPA: {expected_gpa:.2f}")

Course: 20, Expected GPA: 2.66
Course: 50, Expected GPA: 1.92
Course: 25, Expected GPA: 1.72
Course: 26, Expected GPA: 1.54
Course: 5, Expected GPA: 1.48


In [64]:
second_year_students

1      False
2       True
3       True
4       True
5      False
       ...  
96     False
97     False
98     False
99     False
100    False
Length: 100, dtype: bool

In [65]:
second_year_student_id = second_year_students.index[second_year_students].tolist()[0]

In [66]:
second_year_student_id


2